In [8]:
import numpy as np
from scipy.interpolate import interp1d

In [ ]:
class MaterialU235:
    def __init__(self):
        # --- Aprox Data (ENDF/B-VIII) ---
        # X: Energía del neutrón [eV]
        self.energy_grid = np.array([
            1e-5,    
            0.0253,  
            0.1,     
            1.0,     
            10.0,
            100.0,
            1000.0, 
            1e4,
            1e5,     
            1e6,     
            2e6,     
            10e6,
            20e6
        ])

        # Y: Efficace section
        # (Sum of fission + Capture + Scattering)
        self.sigma_total_grid = np.array([
            2500.0,  
            698.0,   
            280.0,   
            100.0,   
            60.0,    
            35.0,    
            20.0,    
            15.0,    
            10.0,    
            7.5,     
            7.2,     
            7.5,     
            8.0      
        ])
        
        self.sigma_fission_grid = np.array([
            2100.0,  
            584.0,   
            200.0,   
            40.0,    
            20.0,    
            15.0,    
            7.0,     
            2.5,     
            1.5,     
            1.2,     
            1.3,     
            2.2,     
            2.0      
        ])
        self.interpolador_fission = interp1d(
            np.log(self.energy_grid), 
            np.log(self.sigma_fission_grid), 
            kind='linear', 
            fill_value="extrapolate"
        )

        self.interpolador = interp1d(
            np.log(self.energy_grid), 
            np.log(self.sigma_total_grid), 
            kind='linear', 
            fill_value="extrapolate"
        )

    def get_total_sigma(self, energia_ev):
        """
        input: Energy in eV
        Outpu: Sigma Total in Barns
        """
        log_e = np.log(energia_ev)
        log_sigma = self.interpolador(log_e)
        return np.exp(log_sigma)
    
    def get_sigma_fission(self, energia_ev):
        """
        input: Energy in eV
        Outpu: Sigma fission in Barns
        """

        log_e = np.log(energia_ev)
        log_sigma = self.interpolador_fission(log_e)
        return np.exp(log_sigma)

class MaterialU238:
    def __init__(self):
        # --- Aprox Data (ENDF/B-VIII for U-238) ---
        # X: Energía del neutrón [eV]
        self.energy_grid = np.array([
            1e-5,    
            0.0253,  
            0.1,     
            1.0,     
            10.0,
            100.0,
            1000.0, 
            1e4,
            1e5,     
            1e6,     
            2e6,     
            10e6,
            20e6
        ])

        # Y: Efficace section
        # (Sum of fission + Capture + Scattering)
        # NOTA: U-238 es mucho mas "pequeño" en térmico (~12 barns vs 698 del U-235)
        self.sigma_total_grid = np.array([
            400.0,   # 1e-5: Sube por captura 1/v
            12.0,    # 0.0253: Térmico (Captura ~2.7 + Scattering ~9)
            10.5,    # 0.1
            15.0,    # 1.0: Inicio zona resonancias (promedio)
            25.0,    # 10.0: Resonancias fuertes
            20.0,    # 100.0
            15.0,    # 1000.0
            13.0,    # 1e4
            10.0,    # 1e5
            7.5,     # 1e6: Zona rápida
            7.2,     # 2e6
            7.5,     # 10e6
            7.8      # 20e6
        ])
        
        # NOTA: Fisión es prácticamente CERO hasta llegar a 1 MeV (umbral)
        # Usamos 1e-9 en lugar de 0 para evitar error matemático en log()
        self.sigma_fission_grid = np.array([
            1e-9,    # 1e-5: Nada
            1e-9,    # 0.0253: Nada
            1e-9,    # 0.1
            1e-9,    # 1.0
            1e-9,    # 10.0
            1e-9,    # 100.0
            1e-9,    # 1000.0
            1e-5,    # 1e4: Empieza a despertar muy levemente
            1e-3,    # 1e5
            0.05,    # 1e6: Umbral de fisión (comienza a subir)
            0.55,    # 2e6: Ya fisiona significativamente
            1.0,     # 10e6: Meseta alta
            1.2      # 20e6
        ])
        
        self.interpolador_fission = interp1d(
            np.log(self.energy_grid), 
            np.log(self.sigma_fission_grid), 
            kind='linear', 
            fill_value="extrapolate"
        )

        self.interpolador = interp1d(
            np.log(self.energy_grid), 
            np.log(self.sigma_total_grid), 
            kind='linear', 
            fill_value="extrapolate"
        )

    def get_total_sigma(self, energia_ev):
        """
        input: Energy in eV
        Outpu: Sigma Total in Barns
        """
        
        log_e = np.log(energia_ev)
        log_sigma = self.interpolador(log_e)
        return np.exp(log_sigma)
    
    def get_sigma_fission(self, energia_ev):
        """
        input: Energy in eV
        Outpu: Sigma fission in Barns
        """
        
        log_e = np.log(energia_ev)
        log_sigma = self.interpolador_fission(log_e)
        return np.exp(log_sigma)

In [7]:
def get_watt_sample():
    # 1. Define the domain and PDF function
    # We go from 0 to 15 (the probability after 15 is negligible)
    x = np.linspace(0, 15, 5000) 
    
    C = 0.453
    a = 1.036
    b = 2.29
    pdf = C * np.exp(-a * x) * np.sinh(np.sqrt(b * x))
    
    #2. Calculate the CDF (Cumulative Sum) numerically
    # Because is easier than the analytic solution
    cdf = np.cumsum(pdf)
    # We normalize so that it goes exactly from 0 to 1.
    cdf = cdf / cdf[-1]
    
    # 3. Generar números aleatorios uniformes u entre 0 y 1
    u = np.random.random()
    
    # 4. Invertir la CDF usando interpolación lineal
    # Encontramos qué valor de x corresponde a cada probabilidad u
    sample = np.interp(u, cdf, x)
    return sample

np.float64(1.3927424531967054)

In [24]:
#Energy of a neutron in eV
class Neutron:
    def __init__(self, energy = 0.025):
        self.energy = energy
        
    def get_energy(self):
        return self.energy
    def set_energy(self, energy):
        self.energy = energy
#Energy of a neutron in eV

def simulation(n_initial_neutrons=1000
               , n_generations_max = 1000
               , n_max_neutrons = 1000000
               ,N_U235 = 0.007
               ,N_U238 = 0.993
               ):
    u235 = MaterialU235()
    u238 = MaterialU238()
    neutrons = []
    new_neutrons = []
    for i in range(n_initial_neutrons):
        neutron_instance = Neutron()
        neutrons.append(neutron_instance)
    generation = 0
    while generation <= n_generations_max:
        print(f"Generation {generation}: Number of neutrons = {len(neutrons)}")
        for i in range(len(neutrons)):
            neutron = neutrons[i]
            E = neutron.get_energy()
            #Determine contact probability with U235
            # Total section efficace 
            sec_eff_U235 = u235.get_total_sigma(E) 
            sec_eff_U238 = u238.get_total_sigma(E)
            # Total section efficace macro
            sec_eff_macro_U235 = sec_eff_U235 * N_U235
            sec_eff_macro_U238 = sec_eff_U238 * N_U238
            #Probability to have contact with an atom U235
            p_contact_U235 = sec_eff_macro_U235 / (sec_eff_macro_U235 + sec_eff_macro_U238)
            
            if np.random.random() <= p_contact_U235: 
                #There is a contact, but now there is a fussion?
                sec_eff_fussion_235 = u235.get_sigma_fission(E)
                p_fission = sec_eff_fussion_235/sec_eff_U235
                if np.random.random() <= p_fission:
                    # There is a fission, but now. How many newtrons it produce?
                    # Using the table that is in the document
                    if E <=1:
                        n_mean = 2.43
                        # 43% probability of be 3 and 57% probability of be 2
                    else:
                        n_mean = 2.35
                        # 35% probability of be 3 and 65% probability of be 2
                    if np.random.random() < (n_mean - 2):
                        n_new_neutrons = 3
                    else:
                        n_new_neutrons = 2
                    
                    # Now we know the numbers of neutrons that the fission genered
                    # Now How many energy do they have?
                    # We are going to use the Watt distribution that we put on the document
                    for j in range(n_new_neutrons):
                        new_neutron = Neutron(get_watt_sample()*1e6)# Converting MeV to eV
                        new_neutrons.append(new_neutron)
            # If there is no contact with U235, or if there is no fission, the neutron is lost
        neutrons = new_neutrons
        new_neutrons = []
        generation += 1
        
        
        
        if len(neutrons) == 0:
            print("The reaction has stopped, there are no more neutrons.")
            break
        if len(neutrons) > n_max_neutrons:
            print("The reaction has exploded, too many neutrons.")
            break
    return generation, len(neutrons)
simulation()


Generation 0: Number of neutrons = 1000
Generation 1: Number of neutrons = 574
Generation 2: Number of neutrons = 5
The reaction has stopped, there are no more neutrons.


(3, 0)